<a href="https://colab.research.google.com/github/Shruti0196/Linguify/blob/main/Word_Game.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nltk==3.6.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 19.2 MB/s eta 0:00:00
  Attempting uninstall: nltk
    Found existing installation: nltk 3.8.1
    Uninstalling nltk-3.8.1:
      Successfully uninstalled nltk-3.8.1


In [ ]:
!pip install pygtrie==2.4.2

ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 441, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_i

In [ ]:
import nltk
nltk.download('words')
dictionary = nltk.corpus.words.words()

print(len(dictionary))

236736


[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [ ]:
from nltk.corpus import words
import string
from pygtrie import CharTrie

# Returns list of words from the nltk corpus
def get_dictionary(corpus=words.words()):
    # Remove words greater than 4 letters
    corpus = list(filter(lambda x: len(x) >= 4, corpus))

    # Remove words starting with capital letters because they are likely to be names
    corpus = list(filter(lambda x: x[0] not in string.ascii_uppercase, corpus))

    output = CharTrie()
    for word in corpus:
        output[word] = True

    return output

dictionary = get_dictionary()

print(len(dictionary))
# >>> 209410

print(dictionary.keys()[:10])  # Display the first 10 words in the dictionary
# >>> ['aalii', 'aardvark', 'aardwolf', 'abac', 'abaca', 'abacate', 'abacay', 'abacinate', 'abacination', 'abaciscus']

print(dictionary.keys("mediu"))  # Search the dictionary for words that start with the prefix "mediu"
# >>> ['medium', 'mediumism', 'mediumistic', 'mediumization', 'mediumize', 'mediumship', 'medius']


209410
['aalii', 'aardvark', 'aardwolf', 'abac', 'abaca', 'abacate', 'abacay', 'abacinate', 'abacination', 'abaciscus']
['medium', 'mediumism', 'mediumistic', 'mediumization', 'mediumize', 'mediumship', 'medius']


In [ ]:
# from preprocessing_simple import dictionary

# Returns list of words that start with a particular prefix as well as the next possible letters
def search(dictionary, prefix):
    try:
        possible_words = dictionary.keys(prefix)
    except KeyError:  # No word starting with the prefix exists
        return [], []

    try:
        possible_letters = sorted(list(set([word[len(prefix)] for word in possible_words])))
    except IndexError:  # Prefix is already a word
        return possible_words, []

    return possible_words, possible_letters

# Returns whether the player wins if the word is formed
def get_score(player, word):
    parity = len(word) % 2
    score = parity ^ (player % 2)
    return score

print(search(dictionary, "mediu"))
# >>> (['medium', 'mediumism', 'mediumistic', 'mediumization', 'mediumize', 'mediumship', 'medius'], ['m', 's'])

print(get_score(1, "medium"))
# >>> 1

print(get_score(2, "medium"))
# >>> 0

(['medium', 'mediumism', 'mediumistic', 'mediumization', 'mediumize', 'mediumship', 'medius'], ['m', 's'])
1
0


In [ ]:
def value(prefix, player, players_turn):
    possible_words, possible_letters = search(dictionary, prefix)

    if len(possible_letters) == 0:  # Check if game state is terminal
        return get_score(player, possible_words[0])

    if players_turn:  # Max if it is player's turn
        v = -float("inf")
        for letter in possible_letters:
            v = max(v, value(prefix+letter, player, 0))
    else:  # Min if it is opponent's turn
        v = float("inf")
        for letter in possible_letters:
            v = min(v, value(prefix+letter, player, 1))

    return v

In [ ]:
from functools import wraps
from time import time
# from preprocessing_simple import dictionary


# Decorator to determine how long a function takes to run
def timing(f):
    @wraps(f)
    def wrap(*args, **kwargs):
        start = time()
        result = f(*args, **kwargs)
        end = time()
        print(f'{f.__name__} took {(end-start):.3f}s to run\n')
        return result
    return wrap

In [ ]:


@timing
def main(prefix):
    possible_words, possible_letters = search(dictionary, prefix)

    if len(possible_words) == 0:   # Ensure that the letters typed in can be used to form a word
        print("No word can be formed!\n")
        return

    if len(possible_letters) == 0:  # If prefix is already an actual word
        print(prefix, "is already a word!\n")
        return

    player = (len(prefix)+1) % 2  # Assume that the player who plays the next letter wants to win

    if len(possible_words) == 1:  # If only one possible word can be formed with prefix
        word = possible_words[0]
        print("The only possible word is", word)
        print("You win!\n") if get_score(player, word) == 1 else print("You lose:(\n")
        return

    scores = [value(prefix + letter, player, 0) for letter in possible_letters]
    best_letters = [possible_letters[i] for i in range(len(scores)) if scores[i] == 1]

    if not best_letters:  # If there is no possible way to win, just show all possible letters
        print("There are no winning letters")
        print("Possible letters: ", ", ".join(possible_letters), "\n")
        return

    print("Best letters to pick are:", ", ".join(best_letters), "\n")

main(input("Starting letters: "))
# >>> Starting letters:
# >>> Best letters to pick are: a, e, h, l

Starting letters: m
Best letters to pick are: b, l, p 

main took 0.574s to run



In [ ]:
# from nltk.corpus import words
# import string
# from pygtrie import CharTrie


# # Returns list of words from the nltk corpus
# def get_dictionary(corpus=words.words()):
#     # Remove words greater than 4 letters
#     corpus = list(filter(lambda x: len(x) >= 4, corpus))

#     # Remove words starting with capital letters because they are likely to be names
#     corpus = list(filter(lambda x: x[0] not in string.ascii_uppercase, corpus))

#     output = CharTrie()
#     for word in corpus:
#         output[word] = True

#     return output


# dictionary = get_dictionary()

# print(len(dictionary))
# >>> 209410

# print(dictionary.keys()[:10])  # Display the first 10 words in the dictionary
# >>> ['aalii', 'aardvark', 'aardwolf', 'abac', 'abaca', 'abacate', 'abacay', 'abacinate', 'abacination', 'abaciscus']

# print(dictionary.keys("mediu"))  # Search the dictionary for words that start with the prefix "mediu"
# >>> ['medium', 'mediumism', 'mediumistic', 'mediumization', 'mediumize', 'mediumship', 'medius']

In [ ]:
from functools import wraps
from time import time
# from preprocessing_simple import dictionary


# Decorator to determine how long a function takes to run
def timing(f):
    @wraps(f)
    def wrap(*args, **kwargs):
        start = time()
        result = f(*args, **kwargs)
        end = time()
        print(f'{f.__name__} took {(end-start):.3f}s to run\n')
        return result
    return wrap


# Returns list of words that start with a particular prefix as well as the next possible letters
def search(dictionary, prefix):
    try:
        possible_words = dictionary.keys(prefix)
    except KeyError:  # No word starting with the prefix exists
        return [], []

    try:
        possible_letters = sorted(list(set([word[len(prefix)] for word in possible_words])))
    except IndexError:  # Prefix is already a word
        return possible_words, []

    return possible_words, possible_letters


# Returns whether the player wins if the word is formed
def get_score(player, word):
    parity = len(word) % 2
    score = parity ^ (player % 2)
    return score


# print(search(dictionary, "mediu"))
# >>> (['medium', 'mediumism', 'mediumistic', 'mediumization', 'mediumize', 'mediumship', 'medius'], ['m', 's'])

# print(get_score(1, "medium"))
# >>> 1

# print(get_score(2, "medium"))
# >>> 0

In [ ]:
from nltk.corpus import words
import string
from pygtrie import CharTrie
import json


# Recursively remove words that contain another word as prefix
# def remove_unnecessary_words(corpus, prefix):
#     # Base cases
#     if len(corpus) == 0:  # If there are no words that start with the prefix
#         return []
#     if corpus[0] == "":  # If the prefix is a word
#         return [prefix]

#     new_dictionary = []

#     for i in string.ascii_lowercase:
#         possible_words = list(filter(lambda x: x.startswith(i), corpus))  # Find words that start with the next letter
#         remaining_letters = list(map(lambda x: x[1:], possible_words))  # Get remaining letters for the possible words

#         new_dictionary += remove_unnecessary_words(remaining_letters, prefix + i)

#     return new_dictionary


# Load dictionary file if it exists else create a new one
# def get_dictionary(corpus=words.words()):
#     try:
#         with open("dictionary.txt", "r") as d:
#             corpus = json.load(d)

#     except FileNotFoundError:
#         # Remove words greater than 4 letters
#         corpus = list(filter(lambda x: len(x) >= 4, corpus))

#         # Remove words starting with capital letters because they are likely to be names
#         corpus = list(filter(lambda x: x[0] not in string.ascii_uppercase, corpus))

#         # Remove words that contain another word as a prefix
#         corpus = remove_unnecessary_words(corpus, "")

#         with open("dictionary.txt", "w") as d:
#             json.dump(corpus, d)

#     output = CharTrie()
#     for word in corpus:
#         output[word] = True
#     return output


# dictionary = get_dictionary()
def get_dictionary(corpus=words.words()):
    # Remove words greater than 4 letters
    corpus = list(filter(lambda x: len(x) >= 4, corpus))

    # Remove words starting with capital letters because they are likely to be names
    corpus = list(filter(lambda x: x[0] not in string.ascii_uppercase, corpus))

    output = CharTrie()
    for word in corpus:
        output[word] = True

    return output

dictionary = get_dictionary()

# print(len(dictionary))
# >>> 83111

In [ ]:
from unittest import TestCase
# from preprocessing import *


class Test(TestCase):
    def test_remove_prefix(self):
        self.assertEqual(remove_prefix(['a', 'ab'], ""), ['a'])
        self.assertEqual(remove_prefix(['a', 'ab', 'ac'], ""), ['a'])
        self.assertEqual(remove_prefix(['a', 'ab', 'ac', 'b', 'ba', 'bc'], ""), ['a', 'b'])
        self.assertEqual(remove_prefix(['dog', 'doggy', 'doggie', 'dogbone'], ""), ['dog'])

In [ ]:

def value(prefix, player, players_turn):
    possible_words, possible_letters = search(dictionary, prefix)

    if len(possible_letters) == 0:  # Check if game state is terminal
        return get_score(player, possible_words[0])

    if players_turn:  # Max if it is player's turn
        v = -float("inf")
        for letter in possible_letters:
            v = max(v, value(prefix+letter, player, 0))
    else:  # Min if it is opponent's turn
        v = float("inf")
        for letter in possible_letters:
            v = min(v, value(prefix+letter, player, 1))

    return v

In [ ]:
# from helper_functions import *
# from preprocessing import dictionary


# def value(prefix, player, players_turn):
#     possible_words, possible_letters = search(dictionary, prefix)

#     # Check if game state is terminal, making use of the parity of the remaining words
#     if len(possible_letters) == 0 or len(set(map(lambda x: len(x) % 2, possible_words))) == 1:
#         return get_score(player, possible_words[0])

#     if players_turn:
#         v = -float("inf")
#         for letter in possible_letters:
#             v = max(v, value(prefix+letter, player, 0))
#             if v == 1:  # Alpha-beta pruning!
#                 return v
#     else:
#         v = float("inf")
#         for letter in possible_letters:
#             v = min(v, value(prefix+letter, player, 1))
#             if v == 0:  # Alpha-beta pruning!
#                 return v

#     return v

In [ ]:
# from helper_functions import *
# from preprocessing_simple import dictionary


# def value(prefix, player, players_turn):
#     possible_words, possible_letters = search(dictionary, prefix)

#     if len(possible_letters) == 0:  # Check if game state is terminal
#         return get_score(player, possible_words[0])

#     if players_turn:  # Max if it is player's turn
#         v = -float("inf")
#         for letter in possible_letters:
#             v = max(v, value(prefix+letter, player, 0))
#     else:  # Min if it is opponent's turn
#         v = float("inf")
#         for letter in possible_letters:
#             v = min(v, value(prefix+letter, player, 1))

#     return v

In [ ]:
# from value import *
# from value_simple import *


# import time
@timing
def main(prefix):
    # st=time.time()
    possible_words, possible_letters = search(dictionary, prefix)
    # et=time.time()
    # print(et-st)
    if len(possible_words) == 0:   # Ensure that the letters typed in can be used to form a word
        print("No word can be formed!\n")
        return

    if len(possible_letters) == 0:  # If prefix is already an actual word
        print(prefix, "is already a word!\n")
        return

    player = (len(prefix)+1) % 2  # Assume that the player who plays the next letter wants to win

    if len(possible_words) == 1:  # If only one possible word can be formed with prefix
        word = possible_words[0]
        print("The only possible word is", word)
        print("You win!\n") if get_score(player, word) == 1 else print("You lose:(\n")
        return

    scores = [value(prefix + letter, player, 0) for letter in possible_letters]
    best_letters = [possible_letters[i] for i in range(len(scores)) if scores[i] == 1]

    if not best_letters:  # If there is no possible way to win, just show all possible letters
        print("There are no winning letters")
        print("Possible letters: ", ", ".join(possible_letters), "\n")
        return

    print("Best letters to pick are:", ", ".join(best_letters), "\n")


if __name__ == '__main__':
    while True:
        main(input("Starting letters: "))

# >>> Starting letters:
# >>> Best letters to pick are: a, e, h, l

Starting letters: m
Best letters to pick are: b, l, p 

main took 0.313s to run

Starting letters: me
Best letters to pick are: a, c, g, j, y 

main took 0.070s to run

Starting letters: med
Best letters to pick are: a, u 

main took 0.006s to run

Starting letters: mes
Best letters to pick are: y 

main took 0.008s to run



KeyboardInterrupt: ignored